# Machine Learning Project

## NBA Match Predictions

### Currently working on adding significant features (so far: elo rating and win percentage, FT percentage, FG percentage, FG3 percentage and Rebounds of the last 10 games of both teams). The plan is to add as much as we can before starting feature selection.

In [3]:
import pandas as pd

In [4]:
# Load the dataset
df = pd.read_csv("../data/games.csv")
df

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1610612748,104.0,0.398,0.760,...,23.0,53.0,1610612750,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,20.0,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1610612759,108.0,0.412,0.813,...,28.0,52.0,1610612754,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,33.0,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,32.0,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25791,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
25792,2014-10-06,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
25793,2014-10-06,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
25794,2014-10-05,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


In [5]:
del df["GAME_STATUS_TEXT"] #deleting the most useless column 
df=df.sort_values("GAME_DATE_EST") #we order the games by date in ascending order
df = df.dropna() #let's drop all the rows containing null values
df = df.reset_index(drop=True) #reset the indexes


In [6]:
# Adding Elo rating of each team for each game (basic version we have to improve, 
# we may want to somehow reset the Elo ratings of each team at the beginning of each new season)

def add_elo_ratings(dataset, team1_column, team2_column):
  # Initialize the Elo rating of each team to 1500
  elo_ratings = {team: 1500 for team in dataset[team1_column]}
  
  # Add new columns for the Elo ratings of each team
  dataset['home_team_elo'] = 0
  dataset['away_team_elo'] = 0
  
  # Iterate through the games and update the Elo ratings
  for index, row in dataset.iterrows():
    team1 = row[team1_column]
    team2 = row[team2_column]
    
    # Add the Elo ratings to the dataset
    dataset.at[index, 'home_team_elo'] = elo_ratings[team1]
    dataset.at[index, 'away_team_elo'] = elo_ratings[team2]
    
    # Calculate the Elo rating for each team using the formula
    team1_rating = elo_ratings[team1]
    team2_rating = elo_ratings[team2]
    
    # Update the Elo ratings based on the outcome of the game
    if row['HOME_TEAM_WINS'] == 1:
      team1_rating, team2_rating = update_elo_ratings(team1_rating, team2_rating)
    else:
      team2_rating, team1_rating = update_elo_ratings(team2_rating, team1_rating)
      
    # Update the Elo ratings in the dictionary
    elo_ratings[team1] = team1_rating
    elo_ratings[team2] = team2_rating
    
  return dataset


def update_elo_ratings(winner_rating, loser_rating):
  # Calculate the expected score for each team
  winner_expected = 1 / (1 + 10 ** ((loser_rating - winner_rating) / 400))
  loser_expected = 1 / (1 + 10 ** ((winner_rating - loser_rating) / 400))
  
  # Calculate the new ratings
  k = 32  # The k-factor determines the weight of the game in the rating system
  winner_rating += k * (1 - winner_expected)
  loser_rating += k * (0 - loser_expected)
  
  return winner_rating, loser_rating
 

In [7]:
#running the above function on our dataframe
df = add_elo_ratings(df, 'HOME_TEAM_ID', 'VISITOR_TEAM_ID')


In [8]:
df['elo_difference'] = df['home_team_elo']-df['away_team_elo']



In [9]:
def calculate_win_pct(df):
  # Create a new column for the home team's win percentage
  df['WIN_PCT_10_home'] = 0.0
  # Create a new column for the away team's win percentage
  df['WIN_PCT_10_away'] = 0.0

  # Iterate over the rows of the dataframe
  for i, row in df.iterrows():
    # Get the home team's ID and the away team's ID
    home_team_id = row['HOME_TEAM_ID']
    away_team_id = row['VISITOR_TEAM_ID']
    game_date = row['GAME_DATE_EST']

    # Find the home team's previous 10 matches
    home_team_prev_matches = df[((df['HOME_TEAM_ID'] == home_team_id) | (df['VISITOR_TEAM_ID'] == home_team_id)) & (df['GAME_DATE_EST'] < game_date)].tail(10)
    # Find the away team's previous 10 matches
    away_team_prev_matches = df[((df['HOME_TEAM_ID'] == away_team_id) | (df['VISITOR_TEAM_ID'] == away_team_id)) & (df['GAME_DATE_EST'] < game_date)].tail(10)

    # Calculate the home team's win percentage in the previous 10 matches, or set it to 0.0 if there are not enough previous matches
    if home_team_prev_matches.shape[0] == 10:
     home_team_win_pct = home_team_prev_matches[((home_team_prev_matches['HOME_TEAM_ID'] == home_team_id) & (home_team_prev_matches['HOME_TEAM_WINS'] == 1)) 
                                              | ((home_team_prev_matches['VISITOR_TEAM_ID'] == home_team_id) & (home_team_prev_matches['HOME_TEAM_WINS'] == 0))].shape[0] / 10
    else:
      home_team_win_pct = 0.0

    # Calculate the away team's win percentage in the previous 10 matches, or set it to 0.0 if there are not enough previous matches
    if away_team_prev_matches.shape[0] == 10:
       away_team_win_pct = away_team_prev_matches[((away_team_prev_matches['HOME_TEAM_ID'] == away_team_id) & (away_team_prev_matches['HOME_TEAM_WINS'] == 1)) 
       | ((away_team_prev_matches['VISITOR_TEAM_ID'] == away_team_id) & (away_team_prev_matches['HOME_TEAM_WINS'] == 0))].shape[0] / 10
    else:
      away_team_win_pct = 0.0

     # Set the win percentage for the home team and the away team in the current row
    df.at[i, 'WIN_PCT_10_home'] = home_team_win_pct
    df.at[i, 'WIN_PCT_10_away'] = away_team_win_pct

  # Return the modified dataframe
  return df


df = calculate_win_pct(df)

In [10]:
def get_matches_for_team(df, team_id):
  # Filter the dataframe to only include matches played by a specific team
  team_matches = df[(df['HOME_TEAM_ID'] == team_id) | (df['VISITOR_TEAM_ID'] == team_id)]
  # Return the filtered dataframe
  return team_matches

# Example:
team_id = 1610612742
team_matches = get_matches_for_team(df, team_id)

 #checking if win percentage computation works well
team_matches[['HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'HOME_TEAM_WINS', 'WIN_PCT_10_home', 'WIN_PCT_10_away']].head(30)

,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,WIN_PCT_10_home,WIN_PCT_10_away
0,1610612762,1610612742,1,0.0,0.0
3,1610612742,1610612753,1,0.0,0.0
15,1610612747,1610612742,1,0.0,0.0
28,1610612744,1610612742,0,0.0,0.0
46,1610612742,1610612762,1,0.0,0.0
56,1610612742,1610612748,1,0.0,0.0
68,1610612764,1610612742,1,0.0,0.0
77,1610612761,1610612742,1,0.0,0.0
98,1610612759,1610612742,0,0.0,0.0
113,1610612742,1610612740,1,0.0,0.0


In [11]:
#our data frame with win percenage and elo rating features added
df

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,...,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,home_team_elo,away_team_elo,elo_difference,WIN_PCT_10_home,WIN_PCT_10_away
0,2003-10-05,10300001,1610612762,1610612742,2003,1610612762,90.0,0.457,0.735,0.143,...,0.500,0.250,20.0,38.0,1,1500.000000,1500.000000,0.000000,0.0,0.0
1,2003-10-06,10300002,1610612763,1610612749,2003,1610612763,105.0,0.494,0.618,0.267,...,0.700,0.154,20.0,43.0,1,1500.000000,1500.000000,0.000000,0.0,0.0
2,2003-10-07,10300010,1610612764,1610612752,2003,1610612764,104.0,0.506,0.677,0.455,...,0.852,0.188,19.0,37.0,1,1500.000000,1500.000000,0.000000,0.0,0.0
3,2003-10-07,10300004,1610612742,1610612753,2003,1610612742,99.0,0.488,0.667,0.167,...,0.727,0.350,15.0,34.0,1,1484.000000,1500.000000,-16.000000,0.0,0.0
4,2003-10-07,10300008,1610612756,1610612751,2003,1610612756,79.0,0.333,0.781,0.133,...,0.679,0.250,22.0,47.0,0,1500.000000,1500.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25692,2022-03-12,22101010,1610612762,1610612758,2021,1610612762,134.0,0.558,0.710,0.390,...,0.857,0.394,27.0,33.0,1,1561.932411,1367.116582,194.815829,0.6,0.2
25693,2022-03-12,22101009,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,0.407,...,0.760,0.387,28.0,50.0,0,1600.583143,1542.133102,58.450041,0.8,0.5
25694,2022-03-12,22101008,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,0.400,...,0.696,0.386,27.0,39.0,1,1564.148894,1625.823041,-61.674146,0.3,0.7
25695,2022-03-12,22101006,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,0.429,...,0.824,0.208,19.0,40.0,1,1535.689421,1507.834063,27.855358,0.5,0.3


In [12]:
def calculate_FG_PCT_average(df):
  # Create a new column 
  df['FG_PCT_avg_10_HOME'] = 0.0
  # Create a new column 
  df['FG_PCT_avg_10_AWAY'] = 0.0

  # Iterate over the rows of the dataframe
  for i, row in df.iterrows():
    # Get the home team's ID and the away team's ID
    home_team_id = row['HOME_TEAM_ID']
    away_team_id = row['VISITOR_TEAM_ID']
    game_date = row['GAME_DATE_EST']

    # Find the home team's previous 10 matches
    home_team_prev_matches = df[((df['HOME_TEAM_ID'] == home_team_id) | (df['VISITOR_TEAM_ID'] == home_team_id)) & (df['GAME_DATE_EST'] < game_date)].tail(10)
    # Find the away team's previous 10 matches
    away_team_prev_matches = df[((df['HOME_TEAM_ID'] == away_team_id) | (df['VISITOR_TEAM_ID'] == away_team_id)) & (df['GAME_DATE_EST'] < game_date)].tail(10)

    # Calculate the home team's FG_PCT average in the previous 10 matches, or set it to 0.0 if there are not enough previous matches
    if home_team_prev_matches.shape[0] == 10:
     home_team_FG_pctsum1 = home_team_prev_matches[home_team_prev_matches['HOME_TEAM_ID'] == home_team_id]['FG_PCT_home'].sum()
     home_team_FG_pctsum2 = home_team_prev_matches[home_team_prev_matches['VISITOR_TEAM_ID'] == home_team_id]['FG_PCT_away'].sum()
     home_team_FG_pct_avg =  (home_team_FG_pctsum1 + home_team_FG_pctsum2) / 10
    else:
      home_team_FG_pct_avg = 0.0

    # Calculate the away team's FG_PCT average in the previous 10 matches, or set it to 0.0 if there are not enough previous matches
    if away_team_prev_matches.shape[0] == 10:
       away_team_FG_pctsum1 = away_team_prev_matches[away_team_prev_matches['HOME_TEAM_ID'] == away_team_id]['FG_PCT_home'].sum()
       away_team_FG_pctsum2 = away_team_prev_matches[away_team_prev_matches['VISITOR_TEAM_ID'] == away_team_id]['FG_PCT_away'].sum()
       away_team_FG_pct_avg =  (away_team_FG_pctsum1 + away_team_FG_pctsum2) / 10
       
    else:
      away_team_FG_pct_avg = 0.0

     # Set the FG_PCT average for the home team and the away team in the current row
    df.at[i, 'FG_PCT_avg_10_HOME'] = home_team_FG_pct_avg
    df.at[i, 'FG_PCT_avg_10_AWAY'] = away_team_FG_pct_avg

  # Return the modified dataframe
  return df

In [13]:
calculate_FG_PCT_average(df)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,...,AST_away,REB_away,HOME_TEAM_WINS,home_team_elo,away_team_elo,elo_difference,WIN_PCT_10_home,WIN_PCT_10_away,FG_PCT_avg_10_HOME,FG_PCT_avg_10_AWAY
0,2003-10-05,10300001,1610612762,1610612742,2003,1610612762,90.0,0.457,0.735,0.143,...,20.0,38.0,1,1500.000000,1500.000000,0.000000,0.0,0.0,0.0000,0.0000
1,2003-10-06,10300002,1610612763,1610612749,2003,1610612763,105.0,0.494,0.618,0.267,...,20.0,43.0,1,1500.000000,1500.000000,0.000000,0.0,0.0,0.0000,0.0000
2,2003-10-07,10300010,1610612764,1610612752,2003,1610612764,104.0,0.506,0.677,0.455,...,19.0,37.0,1,1500.000000,1500.000000,0.000000,0.0,0.0,0.0000,0.0000
3,2003-10-07,10300004,1610612742,1610612753,2003,1610612742,99.0,0.488,0.667,0.167,...,15.0,34.0,1,1484.000000,1500.000000,-16.000000,0.0,0.0,0.0000,0.0000
4,2003-10-07,10300008,1610612756,1610612751,2003,1610612756,79.0,0.333,0.781,0.133,...,22.0,47.0,0,1500.000000,1500.000000,0.000000,0.0,0.0,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25692,2022-03-12,22101010,1610612762,1610612758,2021,1610612762,134.0,0.558,0.710,0.390,...,27.0,33.0,1,1561.932411,1367.116582,194.815829,0.6,0.2,0.4729,0.4622
25693,2022-03-12,22101009,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,0.407,...,28.0,50.0,0,1600.583143,1542.133102,58.450041,0.8,0.5,0.4896,0.4421
25694,2022-03-12,22101008,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,0.400,...,27.0,39.0,1,1564.148894,1625.823041,-61.674146,0.3,0.7,0.4875,0.4903
25695,2022-03-12,22101006,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,0.429,...,19.0,40.0,1,1535.689421,1507.834063,27.855358,0.5,0.3,0.4932,0.4741


In [14]:
team_matches = get_matches_for_team(df, team_id)

 #checking if everything works fine
team_matches[['HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'FG_PCT_home', 'FG_PCT_away', 'FG_PCT_avg_10_HOME', 'FG_PCT_avg_10_AWAY',  ]].tail(30)

,HOME_TEAM_ID,VISITOR_TEAM_ID,FG_PCT_home,FG_PCT_away,FG_PCT_avg_10_HOME,FG_PCT_avg_10_AWAY
25247,1610612742,1610612744,0.379,0.407,0.4614,0.4802
25260,1610612745,1610612742,0.425,0.544,0.4623,0.4595
25277,1610612742,1610612741,0.500,0.433,0.4700,0.4917
25299,1610612752,1610612742,0.467,0.410,0.4299,0.4712
25315,1610612763,1610612742,0.382,0.516,0.4715,0.4715
25318,1610612742,1610612753,0.410,0.481,0.4804,0.4329
25341,1610612742,1610612760,0.398,0.434,0.4643,0.4244
25357,1610612742,1610612761,0.487,0.385,0.4572,0.4286
25360,1610612742,1610612756,0.435,0.453,0.4540,0.4738
25383,1610612742,1610612763,0.486,0.374,0.4528,0.4673


In [15]:
#Let's add other features with the same procedure

def calculate_stats(df):
  
  df['FT_PCT_avg_10_HOME'] = 0.0
  df['FT_PCT_avg_10_AWAY'] = 0.0
  df['FG3_PCT_avg_10_HOME'] = 0.0
  df['FG3_PCT_avg_10_AWAY'] = 0.0
  df['REB_avg_10_HOME'] = 0.0
  df['REB_avg_10_AWAY'] = 0.0
  # Iterate over the rows of the dataframe
  for i, row in df.iterrows():
    # Get the home team's ID and the away team's ID
    home_team_id = row['HOME_TEAM_ID']
    away_team_id = row['VISITOR_TEAM_ID']
    game_date = row['GAME_DATE_EST']

    # Find the home team's previous 10 matches
    home_team_prev_matches = df[((df['HOME_TEAM_ID'] == home_team_id) | (df['VISITOR_TEAM_ID'] == home_team_id)) & (df['GAME_DATE_EST'] < game_date)].tail(10)
    # Find the away team's previous 10 matches
    away_team_prev_matches = df[((df['HOME_TEAM_ID'] == away_team_id) | (df['VISITOR_TEAM_ID'] == away_team_id)) & (df['GAME_DATE_EST'] < game_date)].tail(10)

    # Calculate the home team's FG_PCT average in the previous 10 matches, or set it to 0.0 if there are not enough previous matches
    if home_team_prev_matches.shape[0] == 10:
     home_team_FT_pctsum1 = home_team_prev_matches[home_team_prev_matches['HOME_TEAM_ID'] == home_team_id]['FT_PCT_home'].sum()
     home_team_FT_pctsum2 = home_team_prev_matches[home_team_prev_matches['VISITOR_TEAM_ID'] == home_team_id]['FT_PCT_away'].sum()
     home_team_FT_pct_avg =  (home_team_FT_pctsum1 + home_team_FT_pctsum2) / 10
     home_team_FG3_pctsum1 = home_team_prev_matches[home_team_prev_matches['HOME_TEAM_ID'] == home_team_id]['FG3_PCT_home'].sum()
     home_team_FG3_pctsum2 = home_team_prev_matches[home_team_prev_matches['VISITOR_TEAM_ID'] == home_team_id]['FG3_PCT_away'].sum()
     home_team_FG3_pct_avg =  (home_team_FG3_pctsum1 + home_team_FG3_pctsum2) / 10
     home_team_REB_sum1 = home_team_prev_matches[home_team_prev_matches['HOME_TEAM_ID'] == home_team_id]['REB_home'].sum()
     home_team_REB_sum2 = home_team_prev_matches[home_team_prev_matches['VISITOR_TEAM_ID'] == home_team_id]['REB_away'].sum()
     home_team_REB_avg =  (home_team_REB_sum1 + home_team_REB_sum2) / 10
    else:
      home_team_FT_pct_avg = 0.0
      home_team_FG3_pct_avg = 0.0
      home_team_REB_avg = 0.0
    # Calculate the away team's FG_PCT average in the previous 10 matches, or set it to 0.0 if there are not enough previous matches
    if away_team_prev_matches.shape[0] == 10:
       away_team_FT_pctsum1 = away_team_prev_matches[away_team_prev_matches['HOME_TEAM_ID'] == away_team_id]['FT_PCT_home'].sum()
       away_team_FT_pctsum2 = away_team_prev_matches[away_team_prev_matches['VISITOR_TEAM_ID'] == away_team_id]['FT_PCT_away'].sum()
       away_team_FT_pct_avg =  (away_team_FT_pctsum1 + away_team_FT_pctsum2) / 10
       away_team_FG3_pctsum1 = away_team_prev_matches[away_team_prev_matches['HOME_TEAM_ID'] == away_team_id]['FG3_PCT_home'].sum()
       away_team_FG3_pctsum2 = away_team_prev_matches[away_team_prev_matches['VISITOR_TEAM_ID'] == away_team_id]['FG3_PCT_away'].sum()
       away_team_FG3_pct_avg =  (away_team_FG3_pctsum1 + away_team_FG3_pctsum2) / 10
       away_team_REB_sum1 = away_team_prev_matches[away_team_prev_matches['HOME_TEAM_ID'] == away_team_id]['REB_home'].sum()
       away_team_REB_sum2 = away_team_prev_matches[away_team_prev_matches['VISITOR_TEAM_ID'] == away_team_id]['REB_away'].sum()
       away_team_REB_avg =  (away_team_REB_sum1 + away_team_REB_sum2) / 10
    else:
      away_team_FT_pct_avg = 0.0
      away_team_FG3_pct_avg = 0.0
      away_team_REB_avg = 0.0
     # Set the FG_PCT average for the home team and the away team in the current row
    df.at[i, 'FT_PCT_avg_10_HOME'] = home_team_FT_pct_avg
    df.at[i, 'FT_PCT_avg_10_AWAY'] = away_team_FT_pct_avg
    df.at[i, 'FG3_PCT_avg_10_HOME'] = home_team_FG3_pct_avg
    df.at[i, 'FG3_PCT_avg_10_AWAY'] = away_team_FG3_pct_avg
    df.at[i, 'REB_avg_10_HOME'] = home_team_REB_avg
    df.at[i, 'REB_avg_10_AWAY'] = away_team_REB_avg

  # Return the modified dataframe
  return df

In [16]:
calculate_stats(df)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,...,WIN_PCT_10_home,WIN_PCT_10_away,FG_PCT_avg_10_HOME,FG_PCT_avg_10_AWAY,FT_PCT_avg_10_HOME,FT_PCT_avg_10_AWAY,FG3_PCT_avg_10_HOME,FG3_PCT_avg_10_AWAY,REB_avg_10_HOME,REB_avg_10_AWAY
0,2003-10-05,10300001,1610612762,1610612742,2003,1610612762,90.0,0.457,0.735,0.143,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
1,2003-10-06,10300002,1610612763,1610612749,2003,1610612763,105.0,0.494,0.618,0.267,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2,2003-10-07,10300010,1610612764,1610612752,2003,1610612764,104.0,0.506,0.677,0.455,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
3,2003-10-07,10300004,1610612742,1610612753,2003,1610612742,99.0,0.488,0.667,0.167,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
4,2003-10-07,10300008,1610612756,1610612751,2003,1610612756,79.0,0.333,0.781,0.133,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25692,2022-03-12,22101010,1610612762,1610612758,2021,1610612762,134.0,0.558,0.710,0.390,...,0.6,0.2,0.4729,0.4622,0.7560,0.7625,0.3778,0.3189,48.0,41.4
25693,2022-03-12,22101009,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,0.407,...,0.8,0.5,0.4896,0.4421,0.8089,0.7398,0.3268,0.3320,46.3,45.6
25694,2022-03-12,22101008,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,0.400,...,0.3,0.7,0.4875,0.4903,0.7322,0.7724,0.3702,0.3825,41.1,49.2
25695,2022-03-12,22101006,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,0.429,...,0.5,0.3,0.4932,0.4741,0.8121,0.8076,0.3554,0.3378,43.2,41.9


In [17]:
df1 = df[df['GAME_DATE_EST']>'2003-12-31'][['HOME_TEAM_ID', 'VISITOR_TEAM_ID','elo_difference', 'WIN_PCT_10_home','WIN_PCT_10_away', 'FG_PCT_avg_10_HOME', 'FG_PCT_avg_10_AWAY', 'FT_PCT_avg_10_HOME', 'FT_PCT_avg_10_AWAY', 'FG3_PCT_avg_10_HOME', 'FG3_PCT_avg_10_AWAY', 'REB_avg_10_HOME', 'REB_avg_10_AWAY', 'HOME_TEAM_WINS']].reset_index(drop=True)
df1[['elo_difference', 'WIN_PCT_10_home','WIN_PCT_10_away', 'FG_PCT_avg_10_HOME', 'FG_PCT_avg_10_AWAY', 'FT_PCT_avg_10_HOME', 'FT_PCT_avg_10_AWAY', 
    'FG3_PCT_avg_10_HOME', 'FG3_PCT_avg_10_AWAY', 'REB_avg_10_HOME', 'REB_avg_10_AWAY']]

,elo_difference,WIN_PCT_10_home,WIN_PCT_10_away,FG_PCT_avg_10_HOME,FG_PCT_avg_10_AWAY,FT_PCT_avg_10_HOME,FT_PCT_avg_10_AWAY,FG3_PCT_avg_10_HOME,FG3_PCT_avg_10_AWAY,REB_avg_10_HOME,REB_avg_10_AWAY
0,257.325367,0.8,0.2,0.5148,0.4394,0.7587,0.7351,0.3544,0.2750,40.6,42.9
1,-134.426675,0.1,0.4,0.4067,0.4420,0.7705,0.7130,0.2760,0.3007,41.9,41.9
2,120.817319,0.7,0.4,0.4714,0.3849,0.7877,0.7442,0.3743,0.2890,42.8,45.6
3,109.700708,0.5,0.4,0.4314,0.4443,0.7464,0.7519,0.3003,0.3014,40.1,38.2
4,87.185292,0.6,0.5,0.4174,0.4256,0.7805,0.7748,0.3674,0.3434,39.6,44.4
...,...,...,...,...,...,...,...,...,...,...,...
25227,194.815829,0.6,0.2,0.4729,0.4622,0.7560,0.7625,0.3778,0.3189,48.0,41.4
25228,58.450041,0.8,0.5,0.4896,0.4421,0.8089,0.7398,0.3268,0.3320,46.3,45.6
25229,-61.674146,0.3,0.7,0.4875,0.4903,0.7322,0.7724,0.3702,0.3825,41.1,49.2
25230,27.855358,0.5,0.3,0.4932,0.4741,0.8121,0.8076,0.3554,0.3378,43.2,41.9


In [18]:
# Feature importance for selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X = df1[['elo_difference', 'WIN_PCT_10_home','WIN_PCT_10_away', 'FG_PCT_avg_10_HOME', 'FG_PCT_avg_10_AWAY', 'FT_PCT_avg_10_HOME', 'FT_PCT_avg_10_AWAY', 
        'FG3_PCT_avg_10_HOME', 'FG3_PCT_avg_10_AWAY', 'REB_avg_10_HOME', 'REB_avg_10_AWAY']]
y = df1['HOME_TEAM_WINS']
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest classifier on the training data
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Get the feature importances
importances = clf.feature_importances_

# Sort the feature importances in descending order
sorted_importances = sorted(importances, reverse=True)

# Print the feature names and importances
for feature, importance in zip(X.columns, sorted_importances):
    print(f"{feature}: {importance:.2f}")

elo_difference: 0.17
WIN_PCT_10_home: 0.10
WIN_PCT_10_away: 0.09
FG_PCT_avg_10_HOME: 0.09
FG_PCT_avg_10_AWAY: 0.09
FT_PCT_avg_10_HOME: 0.09
FT_PCT_avg_10_AWAY: 0.09
FG3_PCT_avg_10_HOME: 0.08
FG3_PCT_avg_10_AWAY: 0.08
REB_avg_10_HOME: 0.05
REB_avg_10_AWAY: 0.05


In [19]:
#Feature Scaling
features = ['elo_difference', 'WIN_PCT_10_home','WIN_PCT_10_away', 'FG_PCT_avg_10_HOME', 
            'FG_PCT_avg_10_AWAY', 'FT_PCT_avg_10_HOME', 
            'FT_PCT_avg_10_AWAY', 'FG3_PCT_avg_10_HOME', 'FG3_PCT_avg_10_AWAY', 'REB_avg_10_HOME', 'REB_avg_10_AWAY']
from sklearn.preprocessing import StandardScaler

# Separate the features and the target
X = df1[features]
y = df1['HOME_TEAM_WINS']

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the features
scaler.fit(X)

# Transform the features using the scaler
X_scaled = scaler.transform(X)

In [20]:
#First try: getting 64% accuracy (upper bound is 70%)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create the Random Forest model
model = RandomForestClassifier(n_estimators=100)

# Fit the model to the training data
model.fit(X_train, y_train)

# Evaluate the model on the test data
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.6510798494154943


## Finding a Baseline: Dummy Classifier

In [21]:
from sklearn.dummy import DummyClassifier

In [23]:
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=0)
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_test, y_test)


0.5934218347533188

The baseline accuracy is about **60%**